# CC 2003

- model: l1, l2, rbf
- n_bkps: ??
- jump: ??
- min_size: ??
- custom_cost: ??
- params: ??

- shape: (n_samples, n_features) OR (n_samples,)

In [9]:
# load libraries
import netCDF4 as nc
import xarray as xr
import glob
import pandas as pd
import os
from itertools import zip_longest
import numpy as np
import seaborn as sns
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy.feature as cf
import ruptures as rpt
from datetime import datetime

In [10]:
data = nc.Dataset('cc_2003.nc')
variable = data['CC']

In [11]:
# Flatten 3D array of Closeness Centrality into 1D array with shape (n_samples,)
def flatten(data):
    flat_data = []
    for i in range(data.shape[2]):
        for j in range(data.shape[1]):
            for k in range(data.shape[0]):
                flat_data.append(data[k][j][i])
    return np.array(flat_data)

# Get the 3D index of the change points
def get_3d_index(arr, var_shape):
    index = []
    for i in range(len(arr)-1):
        ind = np.unravel_index(arr[i], var_shape)
        index.append(ind)
    return index

# Return the time, lat, and lon values related to the change points
def get_coords(index, ds):
    time = []
    lat = []
    lon = []
    for i in range(len(index)):
        t = ds['time'].values[index[i][0]]
        t = t.astype('datetime64[D]').astype(str)
        la = ds['lat'].values[index[i][1]]
        lo = ds['lon'].values[index[i][2]]
        time.append(t)
        lat.append(la)
        lon.append(lo)
    return time, lat, lon

In [12]:
input = flatten(np.array(variable))

# Basic models

## L1

In [7]:
bkps_l1 = rpt.BottomUp('l1').fit_predict(input, n_bkps=4)
print(bkps_l1)

[2011725, 2013460, 2025085, 2026855, 3589860]


In [ ]:
bkps_l1_v2 = rpt.BottomUp('l1', jump=2).fit_predict(input, n_bkps=4)
print(bkps_l1_v2)

## L2

In [8]:
bkps_l2 = rpt.BottomUp('l2').fit_predict(input, n_bkps=4)
print(bkps_l2)

[1018500, 1677230, 2026855, 3082940, 3589860]


## rbf

Not possible, kernel gets killed

In [6]:
# CC data 2003
file_pattern = '../../private/complex_network_coefficients/2000-2009_run_20240105_1808/rasterfiles/Europe/2003/CN_Europe_0.25x0.25deg_CC_2003-06*.nc'
file_paths = glob.glob(file_pattern)

# Open and concatenate files
cc_2003_06 = xr.open_mfdataset(file_paths)

# Rename coefficient
cc_2003_06 = cc_2003_06.rename({'coefficient': 'CC'})

# Save the merged data to a new NetCDF file
if os.path.exists("cc_2003_06.nc"):
    os.remove("cc_2003_06.nc")
    cc_2003_06.to_netcdf("cc_2003_06.nc")
else:
    cc_2003_06.to_netcdf("cc_2003_06.nc")

In [7]:
data = nc.Dataset('cc_2003_06.nc')
variable = data['CC']
input = flatten(np.array(variable))

In [ ]:
bkps_rbf = rpt.BottomUp('rbf').fit_predict(input, n_bkps=4)

In [11]:
ds = xr.open_dataset('cc_2003.nc')
index_l1 = get_3d_index(bkps_l1, variable.shape)
time_l1, lat_l1, lon_l1 = get_coords(index_l1, ds)

In [12]:
change_point_data_l1 = {'Time': time_l1, 'Latitude': lat_l1, 'Longitude': lon_l1}
df_l1 = pd.DataFrame(change_point_data_l1)

In [13]:
print(df_l1)

         Time   Latitude  Longitude
0  2003-07-23  44.097141   3.359550
1  2003-07-23  45.605713  35.228466
2  2003-07-23  56.668571  -6.677903
3  2003-07-23  58.177143  33.973782


In [14]:
index_l2 = get_3d_index(bkps_l2, variable.shape)
time_l2, lat_l2, lon_l2 = get_coords(index_l2, ds)

In [15]:
change_point_data_l2 = {'Time': time_l2, 'Latitude': lat_l2, 'Longitude': lon_l2}
df_l2 = pd.DataFrame(change_point_data_l2)

In [16]:
print(df_l2)

         Time   Latitude  Longitude
0  2003-06-27  69.491432  -0.906367
1  2003-07-14  49.377144  -4.419476
2  2003-07-23  58.177143  33.973782
3  2003-08-19  56.417141   8.127341


In [ ]:
# Load your NetCDF file and variable
data = nc.Dataset('cc_2003.nc')
variable = data['CC']
ds = xr.open_dataset('cc_2003.nc')

arr_1D = flatten(np.array(variable))
change_points = bottomup(arr_1D, "l1", 5)
index = get_3d_index(change_points, variable.shape)
time, lat, lon = get_coords(index, ds)